In [49]:
import qminospy
from qminospy.me2 import ME_NLP
import cobra
import cobrame
import pickle

In [50]:
with open('./me_models/final/iJT964-ME.pickle', 'rb') as f:
    me = pickle.load(f)

In [51]:
def solve_me_model(me, max_mu=1., precision=1e-6, min_mu=0, using_soplex=True,
                  compiled_expressions=None, verbosity = 2, mu_fix = False,
                  growth_key='mu'):
    from qminospy.me1 import ME_NLP1
    ## If fixed growth rate, solve as LP
    if mu_fix:
        me_nlp = ME_NLP1(me)
        me_nlp.solvelp(mu_fix)
    else:
        ## 
        if using_soplex:
            from cobrame.solve.algorithms import binary_search
            binary_search(me, min_mu=min_mu, max_mu=max_mu, debug=True, mu_accuracy=precision,
                compiled_expressions=compiled_expressions)
        else:
            # The object containing solveME methods--composite that uses a ME model object 
            me_nlp = ME_NLP1(me, growth_key=growth_key)
            # Use bisection for now (until the NLP formulation is worked out)
            muopt, hs, xopt, cache = me_nlp.bisectmu(precision=precision, mumax=max_mu, verbosity=verbosity)
            try:
                me.solution.f = me.solution.x_dict['biomass_dilution']
            except:
                pass


In [52]:
solve_me_model(me, max_mu = 0.5, min_mu = .05, using_soplex=False, precision = 1e-2)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 133.019908 seconds
Finished substituting S,lb,ub in 3.686620 seconds
Finished makeME_LP in 0.710011 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.25 0.25 1
Finished substituting S,lb,ub in 3.735425 seconds
Finished makeME_LP in 0.702127 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.125 0.125 1
Finished substituting S,lb,ub in 3.614719 seconds
Finished makeME_LP in 0.716447 seconds
Getting MINOS parameters from ME_NLP...
3 0.0625 0.0625 0.125 0.0625 optimal
Finished substituting S,lb,ub in 3.652693 seconds
Finished makeME_LP in 0.721774 seconds
Getting MINOS parameters from ME_NLP...
4 0.09375 0.09375 0.125 0.09375 optimal
Finished substituting S,lb,ub in 4.117332 seconds
Finished makeME_LP in 0.723346 seconds
Getting MINOS parameters from ME_NLP...
5 0.09375 0.09375 0.109375 0.109375 1
Finished substituting S,lb,ub in 3.809529 seconds
Finished makeME_LP in 0.833264 seconds
Gettin